In [ ]:
!pip install Keras-Preprocessing

In [ ]:
import os
import cv2 as cv
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from tensorflow.keras import metrics
from keras.optimizers import Adam
import time 
import pandas as pd 
import keras
from keras.preprocessing import image
from keras.layers import add, Dense, Dropout, Conv2D,MaxPooling2D,UpSampling2D,Input,BatchNormalization, RepeatVector, Reshape
from keras.models import Model
from keras import backend as K
from keras_preprocessing.image import img_to_array

In [ ]:
# Function for loading images
def load_images(path, limit=3000, target_size=(256, 256)):
    images = []
    count = 0
    for image_name in os.listdir(path):
        if count >= limit:
            break
        img_path = os.path.join(path, image_name)
        if not image_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            continue
        img = cv.imread(img_path, cv.IMREAD_COLOR)
        if img is None:
            continue
        img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
        img = cv.resize(img, target_size)
        img = img.astype('float32') / 255.0
        
        img_tensor = tf.convert_to_tensor(img, dtype=tf.float32)
        images.append(img_tensor)
        count += 1

        if count % 1000 == 0:
            print(f"{count} images loaded.")
    
    images_tensor = tf.stack(images)

    return images_tensor


In [ ]:
train_high_path = "/kaggle/input/loli-street-low-light-image-enhancement-of-street/LoLI-Street Dataset/Train/high"  
test_high_path = "/kaggle/input/loli-street-low-light-image-enhancement-of-street/LoLI-Street Dataset/Val/high"
train_low_path = "/kaggle/input/loli-street-low-light-image-enhancement-of-street/LoLI-Street Dataset/Train/low"
test_low_path = "/kaggle/input/loli-street-low-light-image-enhancement-of-street/LoLI-Street Dataset/Val/low"
low = '/kaggle/input/testing/Screenshot 2024-12-31 141043.png'
high = '/kaggle/input/testing/Screenshot 2024-12-31 140950.png'

In [ ]:
# #This is for loading train images pairs
# x_train = load_images(train_low_path)
# y_train = load_images(train_high_path)

# print(len(x_train))
# print(len(y_train))

# # I have commented this out as i have saved the weights once so no need to load training images again

In [ ]:
# Function to visualize the images and ground truth
def plot_images(image, ground_truth):
    plt.figure(figsize=(7,7))
    
    plt.subplot(1,2,1)
    plt.title('Image', color='green', fontsize=10)
    plt.imshow(image)
    plt.axis('off')  
    
    plt.subplot(1,2,2)
    plt.title('Ground Truth', color='black', fontsize=10)
    plt.imshow(ground_truth)
    plt.axis('off')  
    
    plt.show()

# Function to display some random images from the dataset
def visualize_random_images(x_train, y_train, num_images=5):
    indices = np.random.choice(len(x_train), num_images) 
    for idx in indices:
        image = x_train[idx]
        ground_truth = y_train[idx]
        plot_images(image, ground_truth)


In [ ]:
# Visualize 5 random images from x_train and y_train
visualize_random_images(x_train, y_train, num_images=10)


In [ ]:
import tensorflow as tf

if tf.config.list_physical_devices('GPU'):
    print("GPU is available.")
else:
    print("GPU is not available. Using CPU.")


In [ ]:
# from keras import layers
# def down(filters , kernel_size, apply_batch_normalization = True):
#     downsample = tf.keras.models.Sequential()
#     downsample.add(layers.Conv2D(filters,kernel_size,padding = 'same', strides = 2))
#     if apply_batch_normalization:
#         downsample.add(layers.BatchNormalization())
#     downsample.add(keras.layers.LeakyReLU())
#     return downsample

# def up(filters, kernel_size, dropout = False):
#     upsample = tf.keras.models.Sequential()
#     upsample.add(layers.Conv2DTranspose(filters, kernel_size,padding = 'same', strides = 2))
#     if dropout:        
# #         upsample.dropout(0.2)
#         upsample.add(Dropout(0.1))
#     upsample.add(keras.layers.LeakyReLU())
#     return upsample

In [ ]:
# def model():
#     inputs = layers.Input(shape= [256,256,3])
#     d1 = down(128,(3,3),False)(inputs)
#     d2 = down(128,(3,3),False)(d1)
#     d3 = down(256,(3,3),True)(d2)
#     d4 = down(512,(3,3),True)(d3)
    
#     d5 = down(512,(3,3),True)(d4)

#     u1 = up(512,(3,3),True)(d5)
#     u1 = layers.concatenate([u1,d4])
#     u2 = up(256,(3,3),True)(u1)
#     u2 = layers.concatenate([u2,d3])
#     u3 = up(128,(3,3),True)(u2)
#     u3 = layers.concatenate([u3,d2])
#     u4 = up(128,(3,3),True)(u3)
#     u4 = layers.concatenate([u4,d1])
#     u5 = up(3,(3,3),True)(u4)
#     u5 = layers.concatenate([u5,inputs])
#     output = layers.Conv2D(3,(2,2),strides = 1, padding = 'same')(u5)
#     return tf.keras.Model(inputs=inputs, outputs=output)

In [ ]:
# model=model()

In [ ]:
# model.summary()

In [ ]:
# model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001), loss = 'mean_absolute_error',
#               metrics = ['acc'])
# model.fit(x_train, y_train, epochs = 60 ,batch_size = 10, verbose = 1, shuffle=True)


In [ ]:
# model.save("/kaggle/working/lightenhancement.h5")

In [ ]:
trained_model = load_model("/kaggle/input/final_enhancement/tensorflow2/default/1/lightenhancement.h5")

In [ ]:
trained_model.summary()

In [ ]:
# Print weights of each layer
for layer in trained_model.layers:
    print(f"Layer: {layer.name}")
    weights = layer.get_weights()
    print(f"Weights: {weights}\n")

In [ ]:
# Incase we want to resume the training to improve accuracy slightly

# trained_model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001), loss = 'mean_absolute_error',
#               metrics = ['acc'])

# trained_model.fit(x_train, y_train, epochs = 60 ,batch_size = 10, verbose = 1, shuffle=True)

In [ ]:
#trained_model.fit(x_train, y_train, epochs = 60 ,batch_size = 10, verbose = 1, shuffle=True)

In [ ]:
#trained_model.save("/kaggle/working/lowlightenhancefinal.h5")

In [ ]:
x_test = load_images(test_low_path)
y_test= load_images(test_high_path)
x_test = np.reshape(x_test, (len(x_test), 256, 256, 3))
y_test = np.reshape(y_test, (len(y_test), 256, 256, 3))


In [ ]:
print(len(x_test))
print(len(y_test))

In [ ]:
test_loss, test_accuracy =trained_model.evaluate(x_test, y_test)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

In [ ]:
# Function to visualize the images (High, Low, and Enhanced)
def show_images(high, low, enhanced):
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 3, 1)
    plt.title("High Exposure Image")
    plt.imshow(high)
    plt.axis('off')  

    plt.subplot(1, 3, 2)
    plt.title("Low Exposure Image")
    plt.imshow(low)
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.title("Enhanced Image")
    plt.imshow(enhanced)
    plt.axis('off')

    plt.show()

# Visualizing 10 random samples from test dataset
for _ in range(20):
    random_index = np.random.randint(0, len(x_test))  
    low_sample = x_test[random_index]  
    high_sample = y_test[random_index]  
    enhanced_sample = trained_model.predict(np.expand_dims(low_sample, axis=0))[0]  

    # Visualize the high, low, and enhanced images
    show_images(high_sample, low_sample, enhanced_sample)


In [ ]:

img = cv.imread("/kaggle/input/testing/Screenshot 2024-12-31 141043.png", cv.IMREAD_COLOR)
img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
img = cv.resize(img, (256,256))
img = img.astype('float32') / 255.0

img_tensortest = tf.convert_to_tensor(img, dtype=tf.float32)

In [ ]:
enhanced_sample = trained_model.predict(np.expand_dims(img_tensortest, axis=0))[0]  

show_images(img_tensortest, img_tensortest, enhanced_sample)


In [ ]:
img = cv.imread("/kaggle/input/testing/test_img.png", cv.IMREAD_COLOR)
img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
img = cv.resize(img, (256,256))
img = img.astype('float32') / 255.0

img_tensortruth = tf.convert_to_tensor(img, dtype=tf.float32)

In [ ]:
enhanced_sample = trained_model.predict(np.expand_dims(img_tensortruth, axis=0))[0]  

show_images(img_tensortruth, img_tensortruth, enhanced_sample)
